# Mapping the sequencing data to the reference Genome

In the first step we need to map our short-read sequencing data to a reference genome.

What we need for this step:
- Data:
    - Short-read from Illumina sequencing (`fastq` files)
    - Reference genome (`fasta` files)
- Tools:
    - Mapper: `minimap2` or `bowtie2`
    - `samtools sort`
    - `samtools index`

## Mapping with `minimap2`

Map the data to the reference genome with `minimap2`. We get a mapping file for each chromosome/plasmid (`.sam`).

In [ ]:
for i in Chr1 Chr2 PlA PlB PlC PlDE; do minimap2 -a $i.fasta ../Sequencing/RawReads.fastq > ../Alignments/Raw/$i.sam; done

Sort the `.sam` files with `samtools sort` (Covert the `.sam` files to `.bam`)

In [ ]:
for i in Chr1 Chr2 PlA PlB PlC PlDE; do samtools sort -o $i.bam $i.sam; done

The `.sam` files need very much space. If the step above runs without errors, we can remove these files to reduce the space usage. 

In [5]:
for i in *.sam; do rm $i; done

Now we need to index the `.bam` files with `samtools index`.

In [6]:
for i in *.bam; do samtools index $i; done

### Filtering the maps (MAPQ >=30)

We want only mappings with high quality for further analysis. With `samtools view` we remove all mappings with a phred quality score smaller than 30. 

In [8]:
for i in Chr1 Chr2 PlA PlB PlC PlDE; do samtools view -q 30 -b Raw/$i.bam > Filtered/$i.q30.bam; done

## Mapping with `bowtie2`

For the mapping with `bowtie2`, we need to create a bowtie index for the reference data with `bowtie2-build`.

In [3]:
for i in Chr1 Chr2 PlA PlB PlC PlDE; do bowtie2-build -f $i.fasta indices/$i; done

Now we can map our reads with `bowtie2`

In [ ]:
cd Data/indices/
for i in Chr1 Chr2 PlA PlB PlC PlDE; do bowtie2 -x $i -f -U Data/Sequencing/RawReads.fasta -S Data/Alignments/bowtie2_$i.sam --threads 10; done

Similar to the mapping with `minimap2`, we need to sort, index and filter our mapping again.

In [ ]:
for i in Chr1 Chr2 PlA PlB PlC PlDE; do samtools sort -o bowtie2_$i.bam bowtie2_$i.sam; done

In [9]:
for i in *.sam; do rm $i; done

In [11]:
for i in *.bam; do samtools index $i; done

In [14]:
for i in Chr1 Chr2 PlA PlB PlC PlDE; do samtools view -q 30 -b bowtie2_$i.bam > ../Filtered/$i.q30.bam; done

With the `samtools depth` command we can get some statistics 

In [15]:
cd ../Filtered; for i in Chr1 Chr2 PlA PlB PlC PlDE; do samtools depth  $i.q30.bam > $i.tab; done

# Variant Calling with `BCFTools`

## With filtered mapping (`minimap2`)

The first mapper for the consensus SNP calling pipeline is `bcftools mpileup` and following `bcftools call`. As Input we can use our filtered alignments (here the alignment from the `minimap2` aligner but the `bowtie2` alignment also works). As we have a bacterial genome we can set the ploidy parameter to 1 (Default is 2).

In [ ]:
for i in Chr1 Chr2 PlA PlB PlC PlDE; do bcftools mpileup -Ou -f $i.fasta ../Alignments/Filtered/$i.q30.bam  | bcftools call -mv --ploidy 1 -Ob -o ../SNPCalls/BCFTools/Filtered/$i.bcf; done

BCFTools outputs a binary `.bcf` file for further analysis we need to convert it into a humand-readable `.vcf` file with `bcftools convert`.

In [17]:
for i in Chr1 Chr2 PlA PlB PlC PlDE; do bcftools convert $i.bcf > $i.vcf; done

We don't need the binary files, so we can remove them.

In [18]:
for i in *.bcf; do rm $i; done

# Variant Calling with `freebayes`

## With filtered mapping (`minimap2`)

With the command `freebayes` we call the variants from the filtered alignment. Similar to the variant calling with `bcftools` we need to set the ploidy parameter to 1. 

In [22]:
for i in Chr1 Chr2 PlA PlB PlC PlDE; do freebayes -f $i.fasta ../Alignments/Filtered/$i.q30.bam --ploidy 1 > ../SNPCalls/Freebayes/Filtered/$i.vcf; done

# Variant Calling with `lofreq`

In [24]:
conda activate lofreq

## With filtered mapping (`minimap2`)

For variant calling with the tool `lofreq` we need to do some preparations. Before variant calling we have to calculate the indel qualities from our filtered alignement with `lofreq indelqual`.

In [33]:
for i in Chr1 Chr2 PlA PlB PlC PlDE; do lofreq indelqual --dindel --ref $i.fasta --out /Data/Alignments/IndelQual/$i.indelqual.bam Data/Alignments/Filtered/$i.q30.bam; done

Now we can call the variants:

In [ ]:
for i in Chr1 Chr2 PlA PlB PlC PlDE; do lofreq call -f /Data/RefGenome/$i.fasta --call-indels -o ~/SnP_Pipeline_full/Data/SNPCalls/Lofreq/Filtered/$i.vcf ~/SnP_Pipeline_full/Data/Alignments/IndelQual/$i.indelqual.bam; done 

In [ ]:
The resulting `.vcf` files will now be merged and annotated. Please refer the notebook `Pipeline`